# Trading Bot

In [ ]:
from datetime import datetime
import os
import time

from apscheduler.schedulers.background import BackgroundScheduler
import pandas as pd
import pandas_ta as ta

from utilities.charting import plot_candlesticks_RSI_chart
from utilities.getData_YahooFinanace import update_daily_data, update_60m_data, update_5m_data
from utilities.telegram_alerts import send_message_to_telegram_group

from strategies.signal_generators import generate_rsi_ema_entry_signals, generate_exit_signals_with_reentry


In [ ]:
# Set Global Variables
tickers = ["RELIANCE.NS","ADANIPORTS.NS","^NSEI"]

### Update Data & Calculate Indicators

In [ ]:
def update_data_and_calculate_indicators(tickers):
    """
    Updates raw daily and intraday price data for the provided tickers,
    then calculates and appends technical indicators to the datasets.

    Indicators calculated:
    - EMA (Exponential Moving Average) with lengths 20 and 200
    - RSI (Relative Strength Index) with lengths 14 and 8

    Saves the processed data with indicators into 'data/indicators/' folder
    as CSVs for multiple timeframes.

    Adds/updates an 'updated_at' column ONLY for newly added rows.

    Parameters:
    - tickers (list of str): List of ticker symbols to update and process.

    Notes:
    - This function assumes that raw data exists or will be created in 'data/raw/'.
    - Uses `pandas_ta` for technical indicator calculations.
    """

    # === Step 1: Update raw data for all tickers ===
    update_daily_data(tickers)
    update_60m_data(tickers)

    # Step 2: For each ticker, compute indicators and save files under 'data/indicators/'
    for ticker in tickers:
        try:
            now_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            # === 60m Data Processing ===
            raw_path_60m = f"data/raw/{ticker}_60m.csv"
            processed_path_60m = f"data/indicators/{ticker}_60m.csv"

            df_60m = pd.read_csv(raw_path_60m, parse_dates=['Datetime'], index_col='Datetime')

            # Calculate technical indicators
            df_60m.ta.ema(length=20, append=True)
            df_60m.ta.ema(length=200, append=True)
            df_60m.ta.rsi(length=14, append=True)
            df_60m.ta.rsi(length=8, append=True)

            # Handle 'updated_at' column for 60m data
            if os.path.exists(processed_path_60m):
                old_df_60m = pd.read_csv(processed_path_60m, parse_dates=['Datetime'], index_col='Datetime')
                last_timestamp_60m = old_df_60m.index[-1]

                # Preserve existing 'updated_at' values if present
                if 'updated_at' in old_df_60m.columns:
                    df_60m['updated_at'] = old_df_60m['updated_at']
                else:
                    df_60m['updated_at'] = None

                # Set 'updated_at' only for new rows
                df_60m.loc[df_60m.index > last_timestamp_60m, 'updated_at'] = now_str
            else:
                # If this is the first time saving the file, set 'updated_at' for all rows
                df_60m['updated_at'] = now_str

            # Save processed intraday data
            df_60m.to_csv(processed_path_60m)
            print(f"✅ 60m indicators added for {ticker}")

            # === Daily Data Processing ===
            raw_path_daily = f"data/raw/{ticker}_D.csv"
            processed_path_daily = f"data/indicators/{ticker}_D.csv"

            df_daily = pd.read_csv(raw_path_daily, parse_dates=['Date'], index_col='Date')

            # Calculate technical indicators
            df_daily.ta.ema(length=20, append=True)
            df_daily.ta.ema(length=200, append=True)
            df_daily.ta.rsi(length=14, append=True)
            df_daily.ta.rsi(length=8, append=True)

            # Mark newly added daily rows with 'updated_at'
            if os.path.exists(processed_path_daily):
                # Load previously saved file to determine the last updated row
                old_df = pd.read_csv(processed_path_daily, parse_dates=['Date'], index_col='Date')
                last_timestamp = old_df.index[-1]

                # If old file had 'updated_at' column, preserve its values
                if 'updated_at' in old_df.columns:
                    df_daily['updated_at'] = old_df['updated_at']
                else:
                    df_daily['updated_at'] = None

                # Set 'updated_at' to now only for new rows
                df_daily.loc[df_daily.index > last_timestamp, 'updated_at'] = now_str
            else:
                # If this is the first time saving the file, set 'updated_at' for all rows
                df_daily['updated_at'] = now_str

            # Save processed daily data
            df_daily.to_csv(processed_path_daily)
            print(f"✅ Daily indicators added for {ticker}")

        except Exception as e:
            print(f"❌ Error processing {ticker}: {e}")

    # for ticker in tickers:
    #     try:
    #         # === 60m Data Processing ===
    #         df_60m = pd.read_csv(f"data/raw/{ticker}_60m.csv", parse_dates=['Datetime'], index_col='Datetime')
    #         df_60m.ta.ema(length=20, append=True)
    #         df_60m.ta.ema(length=200, append=True)
    #         df_60m.ta.rsi(length=14, append=True)
    #         df_60m.ta.rsi(length=8, append=True)
    #         df_60m.to_csv(f"data/indicators/{ticker}_60m.csv")
    #         print(f"✅ 60m indicators added for {ticker}")

    #         # === Daily Data Processing ===
    #         df_daily = pd.read_csv(f"data/raw/{ticker}_D.csv", parse_dates=['Date'], index_col='Date')
    #         df_daily.ta.ema(length=20, append=True)
    #         df_daily.ta.ema(length=200, append=True)
    #         df_daily.ta.rsi(length=14, append=True)
    #         df_daily.ta.rsi(length=8, append=True)
    #         df_daily.to_csv(f"data/indicators/{ticker}_D.csv")
    #         print(f"✅ Daily indicators added for {ticker}")

        # except Exception as e:
        #     print(f"❌ Error processing {ticker}: {e}")

### Generate Signals

In [ ]:
def run_signals_generator(tickers):
    """
    Processes a list of tickers to generate trading signals based on RSI and EMA crossover strategy.

    For each ticker:
    - Loads the 60-minute indicators data from 'data/indicators/{ticker}_60m.csv'.
    - Applies the RSI-EMA signal generator.
    - Saves the resulting DataFrame with signals to 'data/signals/{ticker}_60m.csv'.

    Parameters:
        ticker_list (list of str): List of ticker symbols to process.

    Returns:
        None
    """
    os.makedirs("data/signals", exist_ok=True)  # Ensure output directory exists

    for ticker in tickers:
        try:
            file_path = f"data/indicators/{ticker}_60m.csv"
            df = pd.read_csv(file_path, parse_dates=['Datetime'], index_col='Datetime')

            # Step 1: Generate Entry Signals
            df = generate_rsi_ema_entry_signals(df, rsi_column='RSI_8', ema_column='EMA_20')

            # Step 2: Generate Exit Signals
            df = generate_exit_signals_with_reentry(df, rsi_column='RSI_8', entry_signal_column='Entry Signal')

            # Step 3: Save the DataFrame with both entry & exit signals
            output_path = f"data/signals/{ticker}_60m.csv"
            df.to_csv(output_path)
            print(f"✅ 60m Signals (entry + exit) generated and saved for {ticker}")

        except Exception as e:
            print(f"❌ Signals Generator Failed for {ticker}: {e}")

### Send Alerts to Telegram

In [ ]:
def send_alerts_to_telegram(tickers):
    """
    Sends Telegram alerts for:
    - New trade entries
    - Live MTM updates (every hour)
    - Trade exits (profit booked or stop-loss hit)

    Parameters:
        tickers (list): List of ticker symbols.
    """
    for ticker in tickers:
        try:
            file_path = f"data/signals/{ticker}_60m.csv"
            df = pd.read_csv(file_path, parse_dates=['Datetime'], index_col='Datetime')
            latest_row = df.iloc[-1]

            signal = latest_row.get("Entry Signal")
            trade_status = latest_row.get("Trade_Status")
            exit_reason = latest_row.get("Exit_Reason")
            alert_sent = latest_row.get("Alert_Sent")
            last_close = latest_row.get("Close")

            message = None

            # === Entry Alert ===
            if pd.notnull(signal) and pd.isnull(alert_sent):
                message = (
                    f"📊 *Entry Alert*\n"
                    f"Stock: {ticker}\n"
                    f"Signal: {signal} 📊\n"
                    f"Strategy: RSI-EMA Confirmation\n"
                    f"Go {'📈 Long' if 'Long' in signal else '📉 Short'} at {last_close:.2f}"
                )

            # === Exit Alert ===
            elif trade_status == "Exited" and pd.isnull(alert_sent):
                message = (
                    f"*Exit Alert*\n"
                    f"Stock: {ticker}\n"
                    f"Status: {exit_reason} ✅\n"
                    f"Exit Price: {last_close:.2f}"
                )
            
            # === Exit Alert ===
            elif trade_status == "Exited" and pd.isnull(alert_sent):

                # Choose emoji based on keywords in exit reason
                if "Profit" in exit_reason:
                    exit_emoji = "💰"
                elif "Stopped" in exit_reason:
                    exit_emoji = "🛑"
                else:
                    exit_emoji = "✅"  # default emoji

                message = (
                    f"{exit_emoji} *Exit Alert*\n"
                    f"Stock: {ticker}\n"
                    f"Reason: {exit_reason} {exit_emoji}\n"
                    f"Exit Price: ₹{last_close:.2f}"
                )

            # === Live Trade Status Update ===
            elif isinstance(trade_status, str) and ("Long:" in trade_status or "Short:" in trade_status) and pd.isnull(alert_sent):
                message = (
                    f"⏱ *Live MTM Update*\n"
                    f"Stock: {ticker}\n"
                    f"Status: {trade_status.split(':')[0]}\n"
                    f"MTM: {trade_status.split(':')[1]}\n"
                    f"Last Price: {last_close:.2f}"
                )

            # Send and update if a message was created
            if message:
                send_message_to_telegram_group(message)
                df.at[df.index[-1], "Alert_Sent"] = datetime.now().strftime("%Y-%m-%d %H:%M")
                df.to_csv(file_path)
                print(f"✅ Alert sent for {ticker}")

        except Exception as e:
            print(f"❌ Failed to process {ticker}: {e}")

### Trading Bot Core Function

In [ ]:
from IPython.display import clear_output

def run_trading_bot(tickers: list[str]) -> None:
    """
    Runs the full trading bot workflow:
    1. Updates market data and computes indicators.
    2. Generates trading signals.
    3. Sends alerts to Telegram for valid signals.

    Parameters:
        tickers (list[str]): List of stock tickers to process.
    """
    clear_output(wait=True)  # Clears the notebook cell output
    
    print(f"\n🕒 Trading Bot ran at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    try:
        # Step 1: Update data and calculate indicators
        update_data_and_calculate_indicators(tickers)
    except Exception as e:
        print(f"❌ Failed during data update: {e}")

    try:
        # Step 2: Generate signals
        run_signals_generator(tickers)
    except Exception as e:
        print(f"❌ Failed during signal generation: {e}")

    try:
        # Step 3: Send alerts to Telegram
        send_alerts_to_telegram(tickers)
    except Exception as e:
        print(f"❌ Failed during sending alerts: {e}")

### Scheduled Runs

In [ ]:
def start_scheduler():
    """
    Initializes and starts the APScheduler to run the trading bot at the xxth minute of every hour.
    Keeps the script running indefinitely until interrupted.
    """
    scheduler = BackgroundScheduler()
    scheduler.add_job(run_trading_bot, 'cron', args=[tickers], minute=16, id='trading_bot_job', name='Hourly Trading Bot Job')
    scheduler.start()
    print("⏳ Trading bot will run at the 16th minute of every hour.")

    try:
        # Keep the script running
        while True:
            time.sleep(45)
    except (KeyboardInterrupt, SystemExit):
        print("⛔ Scheduler stopped.")
        scheduler.shutdown()

In [ ]:
start_scheduler()

### Manual Runs

In [ ]:
# Run the complete Trading Bot Workflow
run_trading_bot(tickers)

In [ ]:
# Update the data and calculate indicators
update_data_and_calculate_indicators(tickers)

In [ ]:
# Update the Daily data
update_daily_data(tickers)

In [ ]:
# Update the 60m data
update_60m_data(tickers)

### Charts

In [ ]:
df = pd.read_csv("data/processed/RELIANCE.NS_60m.csv", parse_dates=['Datetime'], index_col='Datetime').tail(200)

In [ ]:
plot_candlesticks_RSI_chart(df)

In [ ]:
plot_candlesticks_RSI_chart(df, 8)